In [21]:
import os, csv, requests
from bs4 import BeautifulSoup

In [22]:
folderPath = './Testdata/'
fileList = os.listdir(folderPath)
csvName = ""
for f in fileList:
    csvName = f

In [23]:
def hasCode(soup, postId):
    code = None
    answer = soup.find(id="answer-"+postId) #first check if the answer still exists (might be removed)
    
    if answer: #then check for a code example in the answer
        fullAnswer = answer.find('div', class_="s-prose js-post-body")
        code = fullAnswer.findAll('pre') #pre tag is only for code blocks
        if code:
            return code
        else:
            return None
    else:
        return None

In [24]:
def is_c_or_cpp(soup, postId):
    answer = dict()
    c_or_cpp = False
    question = soup.find('div', class_='question')
    post_tags = question.find('div',class_='post-taglist').findAll('a', class_='post-tag')
    
    for p in post_tags:
        if p.text == 'c++' or p.text =='c':
            answer[postId] = p.text
            c_or_cpp = True
            break
            
    if c_or_cpp:
        return answer
    else:
        return None

In [25]:
def line_check(code):
    length = code.count('\n')
    if length>5:
        return True
    else:
        return False

In [26]:
with open(folderPath + csvName, newline='') as file:
    csvFile = csv.DictReader(file)
    count = 1
    countAll = 1
    for row in csvFile:
        fileDict = dict(row)
        if fileDict['PostTypeId'] == '2':
            postId = fileDict['PostId']
            #postId = '951046'
            URL = "https://stackoverflow.com/a/"+postId
            page = requests.get(URL)
            print('opening: ' + URL + "    - #" + str(countAll))
            countAll = countAll + 1
            soup = BeautifulSoup(page.content, 'html.parser')
            
            #check if page still exists
            pageFound = soup.find('div', itemprop="mainEntity")
            if pageFound:
                #check if answer is contains a code example
                code = hasCode(soup,postId)
                if code == None:
                    continue
                else:
                    #check if question is related to c or c++ 
                    answerDict = is_c_or_cpp(soup,postId)
                    if answerDict == None:
                        continue
                    else:
                        #output the anwer into c/c++ files
                        print(postId + " contains c or c++ code in it's answer - #" + str(count))
                        count = count + 1
                        i = 1
                        for c in code:
                            if line_check(c.text)==True:
                                output_path = './output'
                                file_name ="post-"+postId
                                if i>1:
                                    file_name = file_name + "-"+str(i)
                                if answerDict[postId] == "c++":
                                    file_name = file_name +".cpp"
                                elif answerDict[postId] == "c":
                                    file_name = file_name +".c"
                                output = os.path.join(output_path, file_name)
                                file1 = open(output, "w")
                                file1.write(c.text)
                                print(file_name+ " created")
                                file1.close()
                                i = i + 1               

opening: https://stackoverflow.com/a/111917    - #1
opening: https://stackoverflow.com/a/164358    - #2
opening: https://stackoverflow.com/a/165827    - #3
opening: https://stackoverflow.com/a/169253    - #4
opening: https://stackoverflow.com/a/169256    - #5
opening: https://stackoverflow.com/a/170972    - #6
opening: https://stackoverflow.com/a/171075    - #7
opening: https://stackoverflow.com/a/171223    - #8
opening: https://stackoverflow.com/a/171232    - #9
opening: https://stackoverflow.com/a/195331    - #10
opening: https://stackoverflow.com/a/195367    - #11
opening: https://stackoverflow.com/a/256397    - #12
opening: https://stackoverflow.com/a/283595    - #13
opening: https://stackoverflow.com/a/283695    - #14
opening: https://stackoverflow.com/a/365747    - #15
opening: https://stackoverflow.com/a/381555    - #16
opening: https://stackoverflow.com/a/394073    - #17
opening: https://stackoverflow.com/a/394319    - #18
opening: https://stackoverflow.com/a/394322    - #19
op

opening: https://stackoverflow.com/a/1200005    - #122
opening: https://stackoverflow.com/a/1205932    - #123
opening: https://stackoverflow.com/a/1217804    - #124
opening: https://stackoverflow.com/a/1240906    - #125
opening: https://stackoverflow.com/a/1252441    - #126
opening: https://stackoverflow.com/a/1252443    - #127
opening: https://stackoverflow.com/a/1252456    - #128
opening: https://stackoverflow.com/a/1252478    - #129
opening: https://stackoverflow.com/a/1252606    - #130
opening: https://stackoverflow.com/a/1282482    - #131
opening: https://stackoverflow.com/a/1307110    - #132
opening: https://stackoverflow.com/a/1352728    - #133
opening: https://stackoverflow.com/a/1354371    - #134
1354371 contains c or c++ code in it's answer - #27
opening: https://stackoverflow.com/a/1366978    - #135
opening: https://stackoverflow.com/a/1367079    - #136
opening: https://stackoverflow.com/a/1372385    - #137
opening: https://stackoverflow.com/a/1374407    - #138
opening: http

opening: https://stackoverflow.com/a/2336801    - #257
opening: https://stackoverflow.com/a/2348134    - #258
opening: https://stackoverflow.com/a/2378796    - #259
opening: https://stackoverflow.com/a/2426539    - #260
opening: https://stackoverflow.com/a/2440747    - #261
opening: https://stackoverflow.com/a/2477700    - #262
opening: https://stackoverflow.com/a/2478130    - #263
opening: https://stackoverflow.com/a/2484860    - #264
opening: https://stackoverflow.com/a/2494122    - #265
opening: https://stackoverflow.com/a/2506938    - #266


KeyboardInterrupt: 